In [22]:
from __future__ import division, print_function
import random
import numpy as np
import pandas as pd

# Sigmoid function on doubles and its vectorized version
def sigmoid_double(z):
    return 1.0/(1.0+np.exp(-z))

sigmoid = np.vectorize(sigmoid_double)


# Derivative of sigmoid for doubles and numpy arrays
def sigmoid_prime_double(z):
    #
    # Return the correct result for the derivative of the sigmoid function.
    #
    # derivative of sigmoid
    return (sigmoid(z) * (1.0 - sigmoid(z)))

sigmoid_prime = np.vectorize(sigmoid_prime_double)

def softmax(w):
    e = np.exp(w - np.amax(w))
    dist = e / np.sum(e)
    return dist


def softmax_prime(w):
    res = np.zeros((w.shape[0],w.shape[0]))
    for rows in range(0,len(w)):
        for cols in range(0,len(w)):
            if rows == cols:
                res[rows][cols] = softmax(w)[rows]*(1- softmax(w)[rows])
            else:
                res[rows][cols] = -softmax(w)[rows]*softmax(w)[cols]
    return res


class Objective(object):

    def cost_function(self, predictions, labels):
        raise NotImplementedError

    def cost_derivative(self, predictions, labels):
        raise NotImplementedError

class MSE(Objective):
    '''
    Mean squared error function and its derivative.
    '''
    def cost_function(self, predictions, labels):
        diff = predictions - labels
        return 0.5 * sum(diff*diff)[0]

    def cost_derivative(self, predictions, labels, typ):
        #
        # 
        # the derivative of the above MSE cost function.
        # 
        predictions = np.array(predictions)
        labels = np.array(labels)
        labels = labels.reshape(predictions.shape[0], predictions.shape[1])
        if predictions.shape != labels.shape:
            print('Shape Mismatch between Predictions and Labels')
            return 
        else:
            return (predictions - labels)
class Crossentropy(Objective):
    '''
    Crossentropy error function and its derivative.
    '''
    def cost_function(self, predictions, labels):
        -sum(labels * np.log(predictions))

    def cost_derivative(self, predictions, labels, typ):
        #
        # the derivative of the above crossentropy cost function.
        # 
        predictions = np.array(predictions)
        labels = np.array(labels)
        labels = labels.reshape(predictions.shape[0], predictions.shape[1])
        if predictions.shape != labels.shape:
            print('Shape Mismatch between Predictions and Labels')
            return 
        else:
            if typ == 'Softmax':
                return (predictions,labels) # Although this derivative is -label/prediction
            elif typ == 'Sigmoid':
                return -labels/predictions
        
class Network():
    '''
    Neural network class, designed to sequentially stack layers in a simple
    fashion.

    As objective / cost function we use mean square error (MSE), which is found
    in the utils module. All layers should be implemented in the layers module.
    The model is trained by mini-batch gradientdescent.

    Usage:
        - Initialize the network by creating a new instance, e.g.
            net = Network()
        - Add layers one by one using the 'add' method, e.g.
            net.add(layers.ActivationLayer(100))
        - Train the model on data by calling the 'train' method:
            net.train(data, num_of_epochs, mini_batch_size,
                      learning_rate, test_data)
    '''

    def __init__(self, objective=None):
        print("Initialize Network...")
        self.layers = []
        if objective is None:
            self.objective = Crossentropy()

    def add(self, layer):
        '''
        Add a layer, connect it to its predecessor and let it describe itself.
            - layer: A layer from the layers module
        '''
        self.layers.append(layer)
        layer.describe()
        if len(self.layers) > 1:
            self.layers[-1].connect(self.layers[-2])

    def train(self, training_data, epochs, mini_batch_size,
              learning_rate, test_data=None):
        '''
            First, shuffle training data, then split it into mini batches.
            Next, for each mini-batch,
            train this batch. In case test data is present, evaluate it.

            - training_data: A numpy array of pairs containing features and
              labels
            - epochs: The number of epochs/iterations to be trained.
            - mini_batch_size: Number of samples to fit in one batch
            - learning_rate: The learning rate for the gradient descent update
              rule
            - test_data: Optional test data for evaluation
        '''
        n_train = len(training_data)
        for j in range(epochs):
            random.shuffle(training_data)
            mini_batches = [training_data[k:k + mini_batch_size]
                            for k in range(0, n_train, mini_batch_size)]
            for mini_batch in mini_batches:
                self.train_batch(mini_batch, learning_rate)
            if test_data:
                n_test = len(test_data)
                print("Accuracy for epoch {0}: {1}".
                      format(j, self.evaluate(test_data) / n_test))
            else:
                print("Epoch {0} complete".format(j))

    def train_batch(self, mini_batch, learning_rate):
        '''
        Feed forward and pass backward a mini-batch, then update parameters
        accordingly.
        '''
        self.forward_backward(mini_batch)
        self.update(mini_batch, learning_rate)

    def update(self, mini_batch, learning_rate):
        '''
        Normalize learning rate, then update each layer.
        Afterwards, clear all deltas to start the next batch properly.
        '''
        learning_rate = learning_rate / len(mini_batch)
        for layer in self.layers:
            layer.updateParams(learning_rate)
        for layer in self.layers:
            layer.clearDeltas()

    def forward_backward(self, mini_batch):
        '''
        For each sample in the mini batch, feed the features forward layer by
        layer.
        Then compute the cost derivative and do layer by layer backpropagation.
        '''
        for x, y in mini_batch:
            self.layers[0].input_data = x
            for layer in self.layers:
                layer.forward()      
            self.layers[-1].input_delta = self.objective.cost_derivative(
            self.layers[-1].output_data, y,self.layers[-1].typ)
            for layer in reversed(self.layers):
                layer.backward()
            # This is the derivative testing code and should be only activated when asked
            '''
            for layer in self.layers:
                if layer.name == 'Dense':
                    tmp_W=layer.params[0][0][0]
                    layer.params[0][0][0]= tmp_W+ 1e-4
                    first_der=self.objective.cost_function(self.single_forward(x),y)
                    layer.params[0][0][0]= tmp_W - 1e-4
                    sec_der=self.objective.cost_function(self.single_forward(x),y)
                    print('The grad check for Dense layers =',(first_der - sec_der)/(2*1e-4))
                    print('The actual derivatives=',np.dot(layer.get_backward_input(),layer.get_forward_input().T)[0][0])
                    layer.params[0][0][0] = tmp_W
            '''
                    

    def single_forward(self, x):
        '''
        Pass a single sample forward and return the result.
        '''
        self.layers[0].input_data = x
        for layer in self.layers:
            layer.forward()
        return self.layers[-1].output_data

    def evaluate(self, test_data):
        '''
        Returns the number of correctly predicted labels.
        '''
        #
        # For each sample in test data compute the outcome produced by the
        # neural net by using 'single_forward'. Then determine the index of the
        # largest element in the outcome vector to compare it to the respective
        # label.
        #
        # For instance, if output_data = (0.0, 0.9, 0.1), then 0.9 is the most
        # likely result of the network and its position in the array should
        # match the label.
        #
        # Return the number of correctly predicted labels.
        count=0
        for x,y in test_data:
            output = self.single_forward(x)
            if np.argmax(output) == np.argmax(y):
                count =  count +1
        return count
    
class Layer(object):
    '''
    A layer in a neural network. A layer knows its predecessor ('previous')
    and its successor ('next'). Each layer has a forward function that
    emits output data from input data and a backward function
    that emits an output delta, i.e. a gradient, from an input delta.
    '''
    def __init__(self):
        self.params = []

        self.previous = None
        self.next = None

        self.output_data = None
        self.output_delta = None

        self.input_data = None
        self.input_delta = None
        
        self.name= None
        
        self.typ = None

    def connect(self, layer):
        '''
        Connect a layer to its neighbours.
        '''
        self.previous = layer
        layer.next = self

    def forward(self):
        '''
        Feed input data forward. Start with:
            data = self.get_forward_input()
        to receive the output of the previous layer.
        The last line should set the output of the layer, i.e. look like
        this:
            self.output_data = output_data_of_this_layer
        '''
        raise NotImplementedError

    def get_forward_input(self): 
        '''
        input_data is reserved for the first layer, all others get their
        input from the previous output.
        '''
        if self.previous is not None:
            return self.previous.output_data
        else:
            return self.input_data

    def backward(self):
        '''
        Similar to the forward pass compute backpropagation of error terms,
        i.e. feed input errors backward by starting with:
            delta = self.get_backward_input()
        At the end, set the error term of this layer like this:
            self.output_delta = output_error_of_this_layer
        '''
        raise NotImplementedError

    def get_backward_input(self):
        '''
        Input delta is reserved for the very last layer, which will be set
        to the derivative of the cost function. All other layers get their
        error terms from their successor.
        '''
        if self.next is not None:
            return self.next.output_delta
        else:
            return self.input_delta
    

class ActivationLayer(Layer):
    '''
    In general, an activation layer computes the activation of neurons in the
    network with some activation function like sigmoid, tanh, etc.
    Here we simply use the sigmoid function for simplicity.
    '''
    def __init__(self, input_dim, typ):
        super(ActivationLayer, self).__init__()
        self.name ='Activation'
        self.input_dim = input_dim
        self.output_dim = input_dim
        self.typ = typ
    def forward(self):
        data = self.get_forward_input()
        if self.typ == 'Sigmoid':
            self.output_data = sigmoid(data)
        elif self.typ == 'Softmax':
            self.output_data = softmax(data)
            
    def backward(self):
        delta = self.get_backward_input()
        data = self.get_forward_input()
        if self.typ == 'Sigmoid':
            self.output_delta = delta * sigmoid_prime(data)
        elif self.typ == 'Softmax':
            self.output_delta =  delta[0] - delta[1]
        
    def clearDeltas(self):
        pass

    def updateParams(self, rate):
        pass

    def describe(self):
        print("|--- " + self.__class__.__name__)
        print("    |--- dimensions: ({0}, {1})"
              .format(str(self.input_dim), str(self.output_dim)))

class DenseLayer(Layer):
    '''
    Classic feed-forward layer. Output is defined as W * x + b.
    '''

    def __init__(self, input_dim, output_dim):

        super(DenseLayer, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.name ='Dense'

        self.weight = np.random.randn(output_dim, input_dim)
        self.bias = np.random.randn(output_dim, 1)
        self.params = [self.weight, self.bias]

        self.delta_w = np.zeros(self.weight.shape)
        self.delta_b = np.zeros(self.bias.shape)

    def forward(self):
        data = self.get_forward_input()
        #
        # the forward computation of the dense layer as sketched below.
        self.output_data = np.zeros((self.output_dim, 1))
        self.output_data = np.dot(self.weight,data) + self.bias
      
    

    def backward(self):
        delta = self.get_backward_input()
        data = self.get_forward_input()
        self.delta_b += delta
        #
        # Calculating the loss der wrt to weights in terms of error from the previous layer (called delta) 
        self.delta_w += np.dot(delta,data.T) #  + .001* self.weight # Adding a regulariser
        #
        # the back propagation step by computing the gradient to be
        # passed to the previous layer. 
        self.output_delta = np.zeros((self.input_dim, 1))
        self.output_delta = np.dot(self.weight.T,delta)

    def clearDeltas(self):
        self.delta_w = np.zeros(self.weight.shape)
        self.delta_b = np.zeros(self.bias.shape)

    def updateParams(self, rate):
        self.weight -= rate * self.delta_w
        self.bias -= rate * self.delta_b

    def describe(self):
        print("|--- " + self.__class__.__name__)
        print("    |--- dimensions: ({0}, {1})"
              .format(str(self.input_dim), str(self.output_dim)))

# Initialize a network and call it 'net'. Add four layers to it as follows.
# First layer: Dense layer with input dimension 784 and output dimension 100
# Second layer: Activation layer of dimension 100
# Third layer: Dense layer with input dimension 100 and output dimension 10
# Fourth layer: Activation layer of dimension 10

net = Network()
net.add(DenseLayer(784,100))
net.add(ActivationLayer(100,'Sigmoid'))
net.add(DenseLayer(100,10))
net.add(ActivationLayer(10,'Softmax'))

# Load train and test data using 'load_mnist'

def encode_label(j):
    e = np.zeros((10, 1))
    e[j] = 1.0
    return e


def shape_data(data, encode=True):
    features = [np.reshape(x, (784, 1)) for x in data[0]]
    if encode:
        labels = [encode_label(y) for y in data[1]]
    else:
        labels = data[1]
    return list(zip(features, labels))

def from_csv_cols(data, num_features=784):
    features = data[:, :num_features]
    labels = data[:, num_features].flatten()
    return (features, labels)

def load_csv_data(train_idx=50000):
    data = pd.read_csv('data.csv', sep=',', header=None)
    np_data = data.values
    raw_train_data = from_csv_cols(np_data[:train_idx, ])
    raw_test_data = from_csv_cols(np_data[train_idx:, ])
    train_data = shape_data(raw_train_data)
    test_data = shape_data(raw_test_data)
    return (train_data, test_data)

train_data, test_data = load_csv_data()
#
# Train the network with train and test data you loaded before. Train 10
# epochs, using a mini-batch size of 10 and a learning rate of 3.0
#
net.train(train_data, epochs=10, mini_batch_size=10,learning_rate=3, test_data=test_data)



Initialize Network...
|--- DenseLayer
    |--- dimensions: (784, 100)
|--- ActivationLayer
    |--- dimensions: (100, 100)
|--- DenseLayer
    |--- dimensions: (100, 10)
|--- ActivationLayer
    |--- dimensions: (10, 10)


/home/sinah/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:417: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Accuracy for epoch 0: 0.8747
Accuracy for epoch 1: 0.925
Accuracy for epoch 2: 0.9382
Accuracy for epoch 3: 0.9257
Accuracy for epoch 4: 0.9373
Accuracy for epoch 5: 0.9495
Accuracy for epoch 6: 0.9453
Accuracy for epoch 7: 0.9484
Accuracy for epoch 8: 0.949
Accuracy for epoch 9: 0.9485
